In [1]:
try:
    import osmnx as ox
except :
    # !pip install --break-system-packages osmnx
    !pip install osmnx
    import osmnx as ox 

In [2]:
# 광진구의 골목길 포함 도로 네트워크 가져오기 (residential, service, tertiary 도로 포함)
place_name = "광진구, 서울, 대한민국"
# graph = ox.graph_from_place(place_name, network_type='drive')
graph = ox.graph_from_place(place_name, network_type='bike')

# 모든 데이터를 수집하는 경우
graph_all = ox.graph_from_place(place_name, network_type='all')


In [3]:
# 모든 도로의 시작점과 종료점 노드 정보
nodes, edges = ox.graph_to_gdfs(graph)

# 모든 데이터 수집
nodes_all, edges_all = ox.graph_to_gdfs(graph_all)


In [4]:
print(f'불러온 nodes : {len(nodes)}')
print(f'불러온 edges : {len(edges)}')

print(f'불러온 nodes_all : {len(nodes_all)}')
print(f'불러온 edges_all : {len(edges_all)}')

불러온 nodes : 3751
불러온 edges : 9639
불러온 nodes_all : 5227
불러온 edges_all : 14461


In [5]:
# edges 데이터프레임의 구조와 첫 몇 개의 행을 확인
print(edges.head())
print(edges.info())

# nodes 데이터프레임의 구조와 첫 몇 개의 행을 확인
print(nodes.head())
print(nodes.info())


                                                       osmid      highway  \
u         v           key                                                   
368702043 4104262755  0    [175320385, 175320419, 175320429]     tertiary   
          8963165265  0                            516647748  residential   
          436866074   0                            516647752  residential   
414683232 11125576999 0                           1199529109    secondary   
          11125576997 0                           1193259435      primary   

                                     name  oneway reversed      length tunnel  \
u         v           key                                                       
368702043 4104262755  0    [광장앞지하차도, 광장로]   False    False  430.542185    yes   
          8963165265  0              워커힐로   False    False  107.043667    NaN   
          436866074   0              워커힐로   False     True  177.203342    NaN   
414683232 11125576999 0              아차산로    True    Fa

In [6]:
try :
    from tqdm import tqdm
except :
    !pip install tqdm
    from tqdm import tqdm

In [7]:
# 전역 배열 초기화
road_names = []
highway_types = []
start_lats = []
start_lons = []
end_lats = []
end_lons = []
road_len = []
tunnel_status = []  
bridge_status = [] 
service_status = []   
road_ids = []
max_speeds = []
lane_counts = []
one_way = []
junction_status = []

# 도로 종류 한글 매핑 딕셔너리
road_type_map = {
    'motorway': '고속도로',
    'trunk': '간선도로',
    'primary': '주도로',
    'secondary': '2차로',
    'tertiary': '3차로',
    'unclassified': '비분류 도로',
    'residential': '주거지 도로',
    'service': '서비스 도로',
    'living_street': '생활도로'
}

# 'edges' 데이터프레임에서 각 도로에 대해 정보 추출

for index, row in tqdm(edges.iterrows(), total=len(edges)): 
    # MultiIndex에서 u와 v 값 추출
    start_node = index[0]  # 시작 노드 (u)
    end_node = index[1]    # 종료 노드 (v)

    # 노드 정보를 기반으로 위도와 경도 가져오기
    try:
        start_lat, start_lon = nodes.loc[start_node, ['y', 'x']]
        end_lat, end_lon = nodes.loc[end_node, ['y', 'x']]
    except KeyError:
        # 노드 정보가 없을 경우 건너뛰기
        continue

    # 도로명(속성 name)과 도로 특성 확인
    road_name = row.get('name', None)  # 도로명이 없는 경우 None 처리
    highway_type = row.get('highway', None)  
    # if highway_type not in ['unclassified', 'motorway', 'trunk', 'primary', 'secondary', 'residential', 'tertiary',
    #                        'living_street', 'primary_link', 'trunk_link', 'secondary_link', 'road'] :
    #     print(highway_type)                                
    tunnel_status_val = row.get('tunnel', None)   
    bridge_status_val = row.get('bridge', None)  # 교량 여부
    service_status_val = row.get('service', None)  # 서비스 도로 여부
    maxspeed = row.get('maxspeed', None)  # 속도 제한
    lanes = row.get('lanes', None)  # 차선 수
    oneway = row.get('oneway', None)  # 일방통행 여부
    junction = row.get('junction', None)  # 교차로 여부
    length = row.get('length', None) 

    # primary_link의 경우 대부분의 도로를 차지 중 -> 문제는 큰도로쪽도 이게 일부 있음
    
    if isinstance(highway_type, list): 
        continue 
        
    if not highway_type:   
        continue

    if highway_type == 'unclassified' :
        continue 
        
    # 국도나 고속도로 등을 제외
    # path를 날려야 등산로가 사라짐
    if highway_type in ['motorway', 'path', 'cycleway', 'road',
                        'trunk', 'trunk_link',
                        'primary', 'primary_link', 
                        'secondary', 'secondary_link']:    
        continue  # 해당 도로는 건너뛰기 
        
        
    # 전역 배열에 데이터 추가
    road_names.append(road_name if road_name else None)  # 값이 없으면 None로 처리
    # 도로 종류를 한글로 매핑
    road_type_in_korean = road_type_map.get(highway_type, None)
    highway_types.append(road_type_in_korean)  # 값이 없으면 None로 처리
    start_lats.append(start_lat if start_lat else None)  # 값이 없으면 None로 처리
    start_lons.append(start_lon if start_lon else None)  # 값이 없으면 None로 처리
    end_lats.append(end_lat if end_lat else None)  # 값이 없으면 None로 처리
    end_lons.append(end_lon if end_lon else None)  # 값이 없으면 None로 처리
    road_len.append(length if length else None)
    tunnel_status.append(tunnel_status_val if tunnel_status_val else None)  # 값이 없으면 None로 처리
    bridge_status.append(bridge_status_val if bridge_status_val else None)  # 값이 없으면 None로 처리
    service_status.append(service_status_val if service_status_val else None)  # 값이 없으면 None로 처리
    road_ids.append(row['osmid'] if row['osmid'] else None)  # 값이 없으면 None로 처리
    max_speeds.append(maxspeed if maxspeed else None)  # 값이 없으면 None로 처리
    lane_counts.append(lanes if lanes else None)  # 값이 없으면 None로 처리
    one_way.append(oneway if oneway else None)  # 값이 없으면 None로 처리
    junction_status.append(junction if junction else None)  # 값이 없으면 None로 처리


  0%|                                                  | 0/9639 [00:00<?, ?it/s]

  6%|██▍                                   | 621/9639 [00:00<00:01, 6208.90it/s]

 13%|████▉                                | 1285/9639 [00:00<00:01, 6457.48it/s]

 20%|███████▍                             | 1941/9639 [00:00<00:01, 6503.42it/s]

 27%|██████████                           | 2614/9639 [00:00<00:01, 6591.92it/s]

...

 75%|███████████████████████████▊         | 7237/9639 [00:01<00:00, 6547.23it/s]

 82%|██████████████████████████████▎      | 7907/9639 [00:01<00:00, 6590.53it/s]

 89%|████████████████████████████████▉    | 8573/9639 [00:01<00:00, 6609.88it/s]

 96%|███████████████████████████████████▍ | 9235/9639 [00:01<00:00, 6581.29it/s]

100%|█████████████████████████████████████| 9639/9639 [00:01<00:00, 6545.39it/s]

In [8]:
# 각 배열의 길이 확인
print(len(road_names), len(highway_types))
print(len(start_lats), len(start_lons))
print(len(end_lats), len(end_lons))

print(f'{len(road_len)}')


8318 8318
8318 8318
8318 8318
8318


In [9]:
average_lat = []
average_lon = []

for i in range(max(len(start_lats), len(end_lats))):
    start_lat = start_lats[i] if i < len(start_lats) else start_lats[-1]
    end_lat = end_lats[i] if i < len(end_lats) else end_lats[-1]
    average_lat.append((start_lat + end_lat) / 2)

for i in range(max(len(start_lons), len(end_lons))):
    start_lon = start_lons[i] if i < len(start_lons) else start_lons[-1]
    end_lon = end_lons[i] if i < len(end_lons) else end_lons[-1]
    average_lon.append((start_lon + end_lon) / 2)


In [10]:
# 데이터프레임 생성
data = {
    '도로명': road_names,
    '도로 종류': highway_types,
    '시작점_위도': start_lats,
    '시작점_경도': start_lons,
    '종료점_위도': end_lats,
    '종료점_경도': end_lons,
    '중앙점_위도' : average_lat,
    '중앙점_경도' : average_lon,
    '도로_길이' : road_len,
    '터널 여부': tunnel_status,
    '교량 여부': bridge_status,
    '서비스 도로 여부': service_status,
    '도로 ID': road_ids,
    '속도 제한': max_speeds,
    '차선 수': lane_counts,
    '일방통행 여부': one_way,
    '교차로 여부': junction_status
}


In [11]:
import pandas as pd

df = pd.DataFrame(data)


In [12]:
try :
    df.to_csv('./make_file/광진구_자동차도로_전체.csv', index=False)
    print("광진구의 도로 정보가 CSV 파일로 저장되었습니다.")
except OSError as e:
    print(e) 

광진구의 도로 정보가 CSV 파일로 저장되었습니다.


## 비교군으로 '전체' 도로 데이터를 불러와 보기

In [13]:
# 전역 배열 초기화
compare_road_names = []
compare_highway_types = []
compare_start_lats = []
compare_start_lons = []
compare_end_lats = []
compare_end_lons = []
compare_road_len = []
compare_tunnel_status = []  
compare_bridge_status = [] 
compare_service_status = []   
compare_road_ids = []
compare_max_speeds = []
compare_lane_counts = []
compare_one_way = []
compare_junction_status = []

# 도로 종류 한글 매핑 딕셔너리
road_type_map = {
    'motorway': '고속도로',
    'trunk': '간선도로',
    'primary': '주도로',
    'secondary': '2차로',
    'tertiary': '3차로',
    'unclassified': '비분류 도로',
    'residential': '주거지 도로',
    'service': '서비스 도로',
    'living_street': '생활도로'
}

# 'edges_all' 데이터프레임에서 각 도로에 대해 정보 추출 
for index, row in tqdm(edges_all.iterrows(), total=len(edges_all)): 
    # MultiIndex에서 u와 v 값 추출
    start_node = index[0]  # 시작 노드 (u)
    end_node = index[1]    # 종료 노드 (v)

    # 노드 정보를 기반으로 위도와 경도 가져오기
    try:
        start_lat, start_lon = nodes_all.loc[start_node, ['y', 'x']]
        end_lat, end_lon = nodes_all.loc[end_node, ['y', 'x']]
    except KeyError:
        # 노드 정보가 없을 경우 건너뛰기
        continue

    # 도로명(속성 name)과 도로 특성 확인
    road_name = row.get('name', None)  # 도로명이 없는 경우 None 처리
    highway_type = row.get('highway', None)  
    # if highway_type not in ['unclassified', 'motorway', 'trunk', 'primary', 'secondary', 'residential', 'tertiary',
    #                        'living_street', 'primary_link', 'trunk_link', 'secondary_link', 'road',
    #                        'path', 'service', 'cycleway'] :
    #     print(highway_type)                                
    tunnel_status_val = row.get('tunnel', None)   
    bridge_status_val = row.get('bridge', None)  # 교량 여부
    service_status_val = row.get('service', None)  # 서비스 도로 여부
    maxspeed = row.get('maxspeed', None)  # 속도 제한
    lanes = row.get('lanes', None)  # 차선 수
    oneway = row.get('oneway', None)  # 일방통행 여부
    junction = row.get('junction', None)  # 교차로 여부
    length = row.get('length', None)   
        
    # primary_link의 경우 대부분의 도로를 차지 중 -> 문제는 큰도로쪽도 이게 일부 있음

    # pass 없이 진행
    if isinstance(highway_type, list):  
        highway_type = highway_type[0]
        
    # if not highway_type:   
    #     continue

    # if highway_type == 'unclassified' :
    #     continue 
        
    # # 국도나 고속도로 등을 제외
    # # path를 날려야 등산로가 사라짐
    # if highway_type in ['motorway', 'path', 'cycleway', 'road',
    #                     'trunk', 'trunk_link',
    #                     'primary', 'primary_link', 
    #                     'secondary', 'secondary_link']:   
    #     continue  # 해당 도로는 건너뛰기 
        
    # 전역 배열에 데이터 추가
    compare_road_names.append(road_name if road_name else None)  # 값이 없으면 None로 처리
    # 도로 종류를 한글로 매핑
    compare_road_type_in_korean = road_type_map.get(highway_type, None)
    compare_highway_types.append(road_type_in_korean)  # 값이 없으면 None로 처리
    compare_start_lats.append(start_lat if start_lat else None)  # 값이 없으면 None로 처리
    compare_start_lons.append(start_lon if start_lon else None)  # 값이 없으면 None로 처리
    compare_end_lats.append(end_lat if end_lat else None)  # 값이 없으면 None로 처리
    compare_end_lons.append(end_lon if end_lon else None)  # 값이 없으면 None로 처리
    compare_road_len.append(length if length else None)
    compare_tunnel_status.append(tunnel_status_val if tunnel_status_val else None)  # 값이 없으면 None로 처리
    compare_bridge_status.append(bridge_status_val if bridge_status_val else None)  # 값이 없으면 None로 처리
    compare_service_status.append(service_status_val if service_status_val else None)  # 값이 없으면 None로 처리
    compare_road_ids.append(row['osmid'] if row['osmid'] else None)  # 값이 없으면 None로 처리
    compare_max_speeds.append(maxspeed if maxspeed else None)  # 값이 없으면 None로 처리
    compare_lane_counts.append(lanes if lanes else None)  # 값이 없으면 None로 처리
    compare_one_way.append(oneway if oneway else None)  # 값이 없으면 None로 처리
    compare_junction_status.append(junction if junction else None)  # 값이 없으면 None로 처리


  0%|                                                 | 0/14461 [00:00<?, ?it/s]

  4%|█▌                                   | 634/14461 [00:00<00:02, 6337.01it/s]

  9%|███▏                                | 1302/14461 [00:00<00:02, 6532.97it/s]

 14%|████▉                               | 1964/14461 [00:00<00:01, 6571.33it/s]

 18%|██████▌                             | 2622/14461 [00:00<00:01, 6508.40it/s]

...

 82%|████████████████████████████▋      | 11863/14461 [00:01<00:00, 6481.80it/s]

 87%|██████████████████████████████▎    | 12528/14461 [00:01<00:00, 6531.21it/s]

 91%|███████████████████████████████▉   | 13189/14461 [00:02<00:00, 6553.57it/s]

 96%|█████████████████████████████████▌ | 13845/14461 [00:02<00:00, 6518.91it/s]

100%|███████████████████████████████████| 14461/14461 [00:02<00:00, 6526.91it/s]

In [14]:
# 각 배열의 길이 확인
print(len(compare_road_names), len(compare_highway_types))
print(len(compare_start_lats), len(compare_start_lons))
print(len(compare_end_lats), len(compare_end_lons))

print(f'{len(road_len)}')


14461 14461
14461 14461
14461 14461
8318


In [15]:
# 데이터프레임 생성
compare_data = {
    '도로명': compare_road_names,
    '도로 종류': compare_highway_types,
    '시작점_위도': compare_start_lats,
    '시작점_경도': compare_start_lons,
    '종료점_위도': compare_end_lats,
    '종료점_경도': compare_end_lons, 
    '도로_길이' : compare_road_len,
    '터널 여부': compare_tunnel_status,
    '교량 여부': compare_bridge_status,
    '서비스 도로 여부': compare_service_status,
    '도로 ID': compare_road_ids,
    '속도 제한': compare_max_speeds,
    '차선 수': compare_lane_counts,
    '일방통행 여부': compare_one_way,
    '교차로 여부': compare_junction_status
}


In [16]:
import pandas as pd

compare_df = pd.DataFrame(compare_data)


In [17]:
try :
    compare_df.to_csv('./make_file/(비교군)광진구_도로_전체.csv', index=False)
    print("광진구의 도로 정보가 CSV 파일로 저장되었습니다.")
except OSError as e:
    print(e) 

광진구의 도로 정보가 CSV 파일로 저장되었습니다.
